# Libraries

In [1]:
%pip install -q -U datasets transformers accelerate sentencepiece

In [2]:
import os
import random
import numpy as np
import torch
import transformers
from pprint import pprint
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments
from sklearn.metrics import f1_score

# Config

In [3]:
seed = 42
lang = 'sun'

# hf_model_id = 'alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20241227113119' # BEST
hf_model_id = 'alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20250111112049'
hf_tokenizer_id = hf_model_id
hf_data_id = 'alxxtexxr/SemEval2025-Task11-Dataset'
# hf_data_config = 'track_a_sun_70_15_15_stratify_v2'
hf_data_config = 'track_a_sun_go_emotions_80_10_10'

In [4]:
def set_seed(seed):
    # Set random seed for NumPy
    np.random.seed(seed)

    # Set random seed for Torch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True  # Ensures deterministic results
    torch.backends.cudnn.benchmark = False  # Avoids non-deterministic algorithms

    # Set random seed for Transformers
    transformers.set_seed(seed)

    # Optionally set random seed for sklearn and Python's own random module
    random.seed(seed)

    # Set random seed for os
    os.environ['PYTHONHASHSEED'] = str(seed)

    print(f"Random seed set to: {seed}")

set_seed(seed)

Random seed set to: 42


# Data

## Load Data

In [5]:
datasets = load_dataset(hf_data_id, hf_data_config)

cols = list(datasets['train'].features)
emotion_cols = [col for col in cols if col not in ['Unnamed: 0', 'text', 'emotion', 'stratify', 'aug_go_emotions']]
splits = [*datasets.keys()]

print("Data columns:", cols)
print("Emotions columns:", emotion_cols)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Data columns: ['text', 'emotion', 'marah', 'jijik', 'takut', 'senang', 'sedih', 'terkejut', 'biasa', 'aug_go_emotions', 'stratify']
Emotions columns: ['marah', 'jijik', 'takut', 'senang', 'sedih', 'terkejut', 'biasa']


In [6]:
class2id = {class_:id for id, class_ in enumerate(emotion_cols)}
id2class = {id:class_ for class_, id in class2id.items()}

print("Class to ID:")
pprint(class2id, width=1)
print()
print("ID to Class:")
pprint(id2class, width=1)

Class to ID:
{'biasa': 6,
 'jijik': 1,
 'marah': 0,
 'sedih': 4,
 'senang': 3,
 'takut': 2,
 'terkejut': 5}

ID to Class:
{0: 'marah',
 1: 'jijik',
 2: 'takut',
 3: 'senang',
 4: 'sedih',
 5: 'terkejut',
 6: 'biasa'}


## Preprocess Data

In [7]:
tokenizer = AutoTokenizer.from_pretrained(hf_tokenizer_id)

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/786k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/445k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.53M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [8]:
def one_hot_encode_emotion(emotion, emotion_cols):
    emotions = emotion.replace(" ", "").split(",")
    one_hot_emotion = [1.0 if emotion_col in emotions else 0.0 for emotion_col in emotion_cols] # Ensure that the label is float, not int
    return one_hot_emotion

def preprocess_function(data):
   text = data['text']
   emotion = data['emotion']
   labels = one_hot_encode_emotion(emotion, emotion_cols)
   data = tokenizer(text, truncation=True)
   data['labels'] = labels
   return data

tokenized_datasets = {split: datasets[split].map(preprocess_function) for split in splits}

Map:   0%|          | 0/3599 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/406 [00:00<?, ? examples/s]

Map:   0%|          | 0/406 [00:00<?, ? examples/s]

In [9]:
# Sanity check
data = tokenized_datasets['train'][5]

print("Text:", data['text'])
print("Emotion(s):", data['emotion'])
print("Labels:", data['labels'], '-->', emotion_cols)

Text: A fiksi ieu lokasi di sekeloa nya?
Emotion(s): senang
Labels: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0] --> ['marah', 'jijik', 'takut', 'senang', 'sedih', 'terkejut', 'biasa']


In [10]:
data_collator = DataCollatorWithPadding(tokenizer)

# Model

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(
    hf_model_id, 
    num_labels=len(emotion_cols),
    id2label=id2class, label2id=class2id,
    problem_type = "multi_label_classification",
)
print(model.classifier)

config.json:   0%|          | 0.00/978 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

RobertaClassificationHead(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (out_proj): Linear(in_features=768, out_features=7, bias=True)
)


# Evaluation

In [12]:
# threshold = 0.975 # BEST: 0.2

def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred, threshold=0.5):
    predictions, labels = eval_pred
    probs = sigmoid(predictions)
    y_pred = (probs > threshold).astype(int)
    y_true = labels.astype(int)

    # Compute F1 score for each type of averaging method
    f1_micro = f1_score(y_true, y_pred, average='micro', zero_division=0.0)
    f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=0.0)
    f1_weighted = f1_score(y_true, y_pred, average='weighted', zero_division=0.0)
    f1_samples = f1_score(y_true, y_pred, average='samples', zero_division=0.0)
    f1_labels = f1_score(y_true, y_pred, average=None, zero_division=0.0)
    f1_labels_dict = {f'f1_label_{emotion_cols[i]}': f1_labels[i] for i in range(len(f1_labels))}

    return {
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
        'f1_weighted': f1_weighted,
        'f1_samples': f1_samples,
        **f1_labels_dict,
    }

In [13]:
for i in range(1, 10):
    threshold = i/10

    train_args = TrainingArguments(
        # Training config
        per_device_train_batch_size=2,
        # num_train_epochs=3,
        num_train_epochs=1,
        learning_rate=2e-5,
        weight_decay=0.01,

        # Logging config for training
        logging_strategy='steps',
        logging_steps=100,

        # Evaluation config during training
        per_device_eval_batch_size=2,
        eval_strategy='steps',
        eval_steps=100,

        # Model saving config
        # output_dir=project_name,
        output_dir=hf_model_id,
        save_strategy='epoch',
        # load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=train_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['val'],
        processing_class=tokenizer,
        data_collator=data_collator,
        compute_metrics=(lambda eval_pred: compute_metrics(eval_pred, threshold)),
    )

    eval = trainer.evaluate(eval_dataset=tokenized_datasets['test'])

    print("="*64)
    print(f"THRESHOLD: {threshold}")
    print("="*64)
    print("Evaluation to copy:")
    f1_keys = [eval_key for eval_key in eval.keys() if 'f1' in eval_key]
    for i, k in enumerate(f1_keys): print(eval[k], end="\t" if i + 1 < len(f1_keys) else "")


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: alimtegar. Use `wandb login --relogin` to force relogin


THRESHOLD: 0.1
Evaluation to copy:
0.7954120274540218	0.7923444976076555	0.7924117231792802	0.8307881773399014	0.783625730994152	0.7393939393939394	0.8235294117647058	0.8571428571428571	0.8	0.7469879518072289	0.8172043010752689

THRESHOLD: 0.2
Evaluation to copy:
0.819618167640381	0.8172484599589322	0.8163439427544155	0.8501642036124794	0.8322981366459627	0.7581699346405228	0.8472222222222222	0.8671328671328671	0.8285714285714286	0.75	0.8539325842696629

THRESHOLD: 0.3
Evaluation to copy:
0.8219686729660022	0.8187633262260128	0.8175270862318511	0.843431855500821	0.8441558441558441	0.7464788732394366	0.851063829787234	0.8652482269503546	0.8059701492537313	0.7571428571428571	0.8837209302325582

THRESHOLD: 0.4
Evaluation to copy:
0.8205496242667315	0.8183807439824945	0.8165703291083389	0.8348932676518884	0.8378378378378378	0.725925925925926	0.8613138686131386	0.8714285714285714	0.803030303030303	0.7737226277372263	0.8705882352941177

THRESHOLD: 0.5
Evaluation to copy:
0.8219416111660612	0.8194905869324474	0.8175312876101986	0.833743842364532	0.8378378378378378	0.7272727272727273	0.8676470588235294	0.8633093525179856	0.8062015503875969	0.7703703703703704	0.8809523809523809

THRESHOLD: 0.6
Evaluation to copy:
0.8121092628853689	0.8108108108108109	0.8083467707044268	0.8193760262725779	0.8137931034482758	0.71875	0.8676470588235294	0.8613138686131386	0.796875	0.7727272727272727	0.8536585365853658

THRESHOLD: 0.7
Evaluation to copy:
0.8042053399036855	0.8027366020524516	0.7994471511364282	0.8095238095238094	0.7801418439716312	0.6991869918699187	0.8592592592592593	0.8613138686131386	0.8031496062992126	0.7727272727272727	0.8536585365853658

THRESHOLD: 0.8
Evaluation to copy:
0.7851522090689171	0.783625730994152	0.7803052117017011	0.7783251231527094	0.7737226277372263	0.6666666666666666	0.8333333333333334	0.8421052631578947	0.7868852459016393	0.7538461538461538	0.8395061728395061

THRESHOLD: 0.9
Evaluation to copy:
0.7511838916470668	0.7518248175182481	0.7484544758633506	0.7200328407224958	0.746268656716418	0.6495726495726496	0.784	0.84375	0.7413793103448276	0.7301587301587301	0.7631578947368421